<a href="https://colab.research.google.com/github/peppefdf/UI_SCP/blob/main/UI_choose_stops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#%pip install osmnx
#%pip install dash
#%pip install dash_leaflet
#%pip install dash_bootstrap_components
#%pip install cplex
#%pip install docplex

import dash
from dash import Dash
import dash_bootstrap_components as dbc
#import dash_html_components as html
from dash import html, callback_context, ALL
from dash import dcc, Output, Input, State, callback
import dash_leaflet as dl
#import re
import json
import pandas as pd
import geopy.distance
import numpy as np

from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
#"/content/drive/MyDrive/Colab Notebooks/calcroutes_module.py"
import calcroutes_module

#"/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/calcroutes_module.py"

im1 = '/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/CSL_logo.PNG'
im2 = '/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/DFG_logo.png'
#im1 = '/home/beppe23/mysite/assets/CSL_logo.PNG'
#im2 = '/home/beppe23/mysite/assets/DFG_logo.png'

stops_file = "/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/GTFS_files_bus_stops_12_02_2024/all_stops_12_02_2024.csv"
#stops_file = "/home/beppe23/mysite/assets/all_stops_12_02_2024.csv"

from PIL import Image
image1 = Image.open(im1)
image2 = Image.open(im2)


stops_df = pd.read_csv(stops_file, encoding='latin-1')

stops_lat_lon = stops_df[['stop_lat','stop_lon']].to_numpy()

center = (43.26852347667122, -1.9741372404905988)
#    iconUrl= 'https://uxwing.com/wp-content/themes/uxwing/download/location-travel-map/bus-stop-icon.png',
#    iconUrl= "https://i.ibb.co/6n1tzcQ/bus-stop.png",
custom_icon = dict(
    iconUrl= "https://i.ibb.co/HV0K5Fp/bus-stop.png",
    iconSize=[40,40],
    iconAnchor=[22, 40]
)

app = Dash(prevent_initial_callbacks=True)

"""
app.layout = html.Div([
    html.Img(src=image1,style={'width':'40%', "display": "inlineBlock", "verticalAlign": "top"}),
    html.Img(src=image2,style={'width':'25%',"display": "inlineBlock", "verticalAlign": "top"}),
    html.Div(dl.Map([dl.TileLayer()],
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})),
    dbc.Button("Load stops", id="load_stops", n_clicks=0),
    html.Div(id='clickdata'),
    dcc.Store(id='internal-value', data=[])
])
"""
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
}

mouse_over_mess = """
Shifts chosen stops to
<p>closest existing bus stops</p>"""

sidebar =  html.Div(
       [
        html.Button("Load stops", id="load_stops", n_clicks=0),
        html.Br(),
        html.Button("Match stops", id="match_stops", n_clicks=0, style={"margin-top": "15px", "font-weight": "bold"}),
        dbc.Popover(dcc.Markdown(mouse_over_mess, dangerously_allow_html=True),
                  target="match_stops",
                  body=True,
                  trigger="hover",style = {'font-size': 12, 'line-height':'2px'}),
        html.P([ html.Br(),'Choose number of buses'],id='buses_num',style={"margin-top": "15px","font-weight": "bold"}),
        dcc.Input(id="choose_buses", type="text", value='3'),
        html.P([ html.Br(),'Liters of gasoline per kilometer'],id='gas_km',style={"margin-top": "15px","font-weight": "bold"}),
        dcc.Input(id="choose_gas_km", type="text", value='1.12'),
        html.P([ html.Br(),'CO2 gr per lt'],id='CO2_lt',style={"margin-top": "15px","font-weight": "bold"}),
        dcc.Input(id="choose_CO2_lt (Kg/lt)", type="text", value='2.3'),
        html.Button("Calculate routes", id="calc_routes", n_clicks=0,style={"margin-top": "15px"}),
        html.P([ html.Br(),'Select route to visualize'],id='route_select',style={"margin-top": "15px", "font-weight": "bold"}),
        dcc.Dropdown(["Route 1", "Route 2", "Route 3"], multi=False,style={"margin-top": "15px"},id='choose_route'),
        html.Br(),
        html.Button("Visualize routes", id="visualize_routes", n_clicks=0,style={"margin-top": "15px"}),
        html.Div(id='outdata'),
        dcc.Store(id='internal-value_stops', data=[]),
        dcc.Store(id='internal-value_routes', data=[])
       ],
       style=SIDEBAR_STYLE)

content = html.Div(
    [
          html.Div([
             html.Img(src=image1,style={'width':'40%', "display": "inlineBlock", "verticalAlign": "top"}),
             html.Img(src=image2,style={'width':'25%',"display": "inlineBlock", "verticalAlign": "top"}),
          ]),
          html.Div([dl.Map([dl.TileLayer()], center=center, zoom=12, id="map",style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
          ])
    ],
    style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


@app.callback([Output("outdata", "children",allow_duplicate=True),Output('internal-value_routes','data'),Output("choose_route", "options")],
              [State('choose_buses',"value")],
              [State('internal-value_stops','data')],
              [Input("calc_routes", "n_clicks")]
              )
def calc_routes(Nroutes,Stops,Nclick):
    list_routes = range(1,int(Nroutes)+1)
    new_menu = [{'label': 'Route ' +str(i), 'value': i} for i in list_routes]
    routes_coords = calcroutes_module.CalcRoutes_module(Stops)
    out=''
    """
    for i in range(len(routes_coords[0])):
        out = out + str(routes_coords[0][i][0]) + '_' + str(routes_coords[0][i][1]) + '; '
    """
    return [str(routes_coords[0][0][1]), routes_coords, new_menu]

#Output('map','children',allow_duplicate=True)

@app.callback([Output('map','children',allow_duplicate=True)],
              [State('choose_route',"value")],
              [State('internal-value_stops','data')],
              [State('internal-value_routes','data')],
              [Input("visualize_routes", "n_clicks")]
              )
def visualize_route(Route,Stops,RoutesCoords,Nclick):
    Route = int(Route)
    RoutesCoords = RoutesCoords[Route]

    markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(Stops)]
    newMap = dl.Map([dl.TileLayer(), dl.Polyline(positions=RoutesCoords)] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [newMap]


@app.callback([Output("outdata", "children",allow_duplicate=True), Output('internal-value_stops','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [State('internal-value_stops','data')],
              [Input("match_stops", "n_clicks")]
              )
def match_stops(St,Nclicks):
    bus_stops = []
    out = ''
    for i_st in range(len(St)):
        #ref = np.array([lat,lon])
        ref = np.array([St[i_st][0],St[i_st][1]])
        ref = np.tile(ref,(len(stops_lat_lon),1)) # generate replicas of ref point
        #d = [sum((p-q)**2)**0.5 for p, q in zip(ref, stops_lat_lon)] # calculate distance of each bus stop to ref point
        d = [geopy.distance.geodesic((p[0],p[1]), (q[0],q[1])).km for p, q in zip(ref, stops_lat_lon)] # calculate distance of each bus stop to ref point

        ind_min = d.index(min(d)) # find index of closest bus stop
        x = stops_lat_lon[ind_min][0]
        y = stops_lat_lon[ind_min][1]
        bus_stops.append((x, y))
        St[i_st]=(x,y)
        out = out + str(St[i_st][0]) + ', ' + str(St[i_st][1]) + '; '
    markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [out,St,newMap]


#@app.callback([Output("clickdata", "children")],
@app.callback([Output("outdata", "children"), Output('internal-value_stops','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [Input("load_stops", "n_clicks")]
              )
def load_stops(N):
    filename = '/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/Accessibility_Map/INPUT_stops.csv'
    df = pd.read_csv(filename)
    #out=St.loc[:'Lat']
    #for i in range(len(St)):
    #    out = out + str(St.loc[i,['Lat']]) + ', ' + str(St.loc[i,['Lon']]) + '; '
    out = ''
    St = []
    for ind in df.index:
         out = out + str(df['Lat'][ind]) + ',' + str(df['Lon'][ind]) +';'
         St.append((df['Lat'][ind],df['Lon'][ind]))
    markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    #return [out,St,newMap]
    return [out,St,newMap]


@app.callback([Output("outdata", "children",allow_duplicate=True), Output('internal-value_stops','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [State('internal-value_stops','data')],
              [Input('map','clickData')]
              )
def add_marker(St,clickd):
    marker_lat = clickd['latlng']['lat']
    marker_lon = clickd['latlng']['lng']
    St.append((marker_lat,marker_lon))
    out=''
    for i in range(len(St)):
        out = out + str(St[i][0]) + ', ' + str(St[i][1]) + '; '
    #markers = [dl.Marker(dl.Tooltip("double click on Marker after drag to update its position"), position=pos, id="marker{}".format(i), draggable=True) for i, pos in enumerate(St)]
    #Layer_group = dl.LayerGroup(markers, id="markers_group")
    #Layer_group = dl.LayerGroup(markers, id={"type": "markers_group", "index": 0})
    #markers = [dl.Marker(id={'type': 'marker', 'index': key}, position=data[key]) for key in data]
    markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [out,St,newMap]

@app.callback([Output("outdata", "children",allow_duplicate=True),Output('internal-value_stops','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [State('internal-value_stops','data')],
              [Input({"type": "marker", "index": ALL},"n_clicks")],
              prevent_initial_callbacks=True)
def remove_marker(St,*args):
    marker_id = callback_context.triggered[0]["prop_id"].split(".")[0].split(":")[1].split(",")[0]
    n_clicks = callback_context.triggered[0]["value"]
    if n_clicks ==2:
       del St[int(marker_id)]
       out=''
       for i in range(len(St)):
           out = out + str(St[i][0]) + ', ' + str(St[i][1]) + '; '
       markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
       newMap = dl.Map([dl.TileLayer()] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
       return [out,St,newMap]


if __name__ == '__main__':
    #app.run_server(Debug=True)
    app.run_server(port=8058,Debug=True)

Mounted at /content/drive


<IPython.core.display.Javascript object>